In [78]:
import sys
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install chart_studio
!{sys.executable} -m pip install seaborn

import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import random

import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py
import chart_studio.tools as tls
import os

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
%env PLOTLY_USER=arhanna
%env PLOTLY_API=6EStzo8mO5ZY6kfaehPe

env: PLOTLY_USER=arhanna
env: PLOTLY_API=6EStzo8mO5ZY6kfaehPe


In [6]:
user = %env PLOTLY_USER
api = %env PLOTLY_API

tls.set_credentials_file(username= user, api_key= api)

# Data Loading Stage + Pre-processing

In [7]:
# Custom dataset
# Added custom column header line to the csv after download
df = pd.read_csv('./data.csv')
df['created_at'] = pd.to_datetime(df['created_at'])
df.describe()

,id,confidence
count,4.990000e+02,499.000000
mean,1.361428e+18,0.714783
std,5.215730e+15,0.128498
min,1.349833e+18,0.500376
25%,1.358297e+18,0.601505
50%,1.363227e+18,0.707502
75%,1.366536e+18,0.823958
max,1.366980e+18,0.997732


In [8]:
df.head()

,text,user,id,created_at,confidence,sentiment
0,RT @Tesla: Model 3 earns Top Safety+ award fro...,elonmusk,1366451032356610050,2021-03-01 18:11:28+00:00,0.652028,Positive
1,@thesheetztweetz Ahem …,elonmusk,1366446908684705792,2021-03-01 17:55:05+00:00,0.839178,Positive
2,https://t.co/jJeP4pFR6m,elonmusk,1365241433381339136,2021-02-26 10:04:57+00:00,0.589783,Positive
3,Do androids dream of electric cars?,elonmusk,1365237858165026818,2021-02-26 09:50:44+00:00,0.504706,Positive
4,@Sharvenium @flcnhvy 🤣🤣,elonmusk,1365226460244561920,2021-02-26 09:05:27+00:00,0.589783,Positive


### Setup groups/counts/sums

In [24]:
# df_s: change sentiment to ints: 1,-1
df_s = df.copy()
# df_s['sentiment'] = df_s['sentiment'].apply(lambda x: 1 if x=="Positive" else -1)
df_s.head()

,text,user,id,created_at,confidence,sentiment
0,RT @Tesla: Model 3 earns Top Safety+ award fro...,elonmusk,1366451032356610050,2021-03-01 18:11:28+00:00,0.652028,Positive
1,@thesheetztweetz Ahem …,elonmusk,1366446908684705792,2021-03-01 17:55:05+00:00,0.839178,Positive
2,https://t.co/jJeP4pFR6m,elonmusk,1365241433381339136,2021-02-26 10:04:57+00:00,0.589783,Positive
3,Do androids dream of electric cars?,elonmusk,1365237858165026818,2021-02-26 09:50:44+00:00,0.504706,Positive
4,@Sharvenium @flcnhvy 🤣🤣,elonmusk,1365226460244561920,2021-02-26 09:05:27+00:00,0.589783,Positive


In [25]:
df_g = df_s.copy()
df_g_sentiment = df_s.groupby(['user', pd.Grouper(key='created_at', freq='D')])['sentiment'].sum().reset_index(name ='sentiment')
df_g.head()
# df_g.describe()

,text,user,id,created_at,confidence,sentiment
0,RT @Tesla: Model 3 earns Top Safety+ award fro...,elonmusk,1366451032356610050,2021-03-01 18:11:28+00:00,0.652028,Positive
1,@thesheetztweetz Ahem …,elonmusk,1366446908684705792,2021-03-01 17:55:05+00:00,0.839178,Positive
2,https://t.co/jJeP4pFR6m,elonmusk,1365241433381339136,2021-02-26 10:04:57+00:00,0.589783,Positive
3,Do androids dream of electric cars?,elonmusk,1365237858165026818,2021-02-26 09:50:44+00:00,0.504706,Positive
4,@Sharvenium @flcnhvy 🤣🤣,elonmusk,1365226460244561920,2021-02-26 09:05:27+00:00,0.589783,Positive


In [26]:
df_g_sentiment.head()

,user,created_at,sentiment
0,benshapiro,2021-03-02 00:00:00+00:00,PositivePositiveNegativePositiveNegativePositi...
1,benshapiro,2021-03-03 00:00:00+00:00,NegativePositiveNegativePositivePositivePositi...
2,elonmusk,2021-02-06 00:00:00+00:00,NegativePositivePositivePositivePositivePositi...
3,elonmusk,2021-02-07 00:00:00+00:00,PositiveNegativeNegativePositivePositivePositi...
4,elonmusk,2021-02-08 00:00:00+00:00,NegativePositivePositivePositivePositiveNegati...


In [41]:
# fig, ax = plt.subplots(figsize=(30,10))
cdm = {"Positive": "Blue", "Negative": "Red"}
co ={"sentiment": ["Positive", "Negative"]}
df_user_sum = df.groupby(['user', 'sentiment'])['sentiment'].count().reset_index(name='count')
fig = px.bar(df_user_sum, x="user", y="count",color="sentiment", title="User Total", category_orders=co)
url = py.plot(fig, filename='user_tweets',auto_open=False)
df_user_sum.head()

,user,sentiment,count
0,benshapiro,Negative,36
1,benshapiro,Positive,64
2,elonmusk,Negative,48
3,elonmusk,Positive,152
4,sza,Negative,78


### Specific User analysis

In [94]:
user = random.choice(df.user.unique())
df_u = df_s[df_s['user']==user].groupby(['user', 'sentiment',pd.Grouper(key='created_at', freq='D')])['sentiment'].count().reset_index(name='total')
df_u['nroll_7'] = df_u[df_u['sentiment']=='Negative'].total.rolling(7,win_type='triang').mean()
df_u['proll_7'] = df_u[df_u['sentiment']=='Positive'].total.rolling(7,win_type='triang').mean()

df_u['nroll_7'] = df_u['nroll_7'].fillna(0)
df_u['proll_7'] = df_u['proll_7'].fillna(0)

# df_u['sentiment'] = df_u['sum'].apply(lambda x: "Negative" if x < 0 else "Positive")
df_u.head(20)
df_u[df_u['sentiment']=='Positive'].head(30)

,user,sentiment,created_at,total,nroll_7,proll_7
31,sza,Positive,2021-01-14 00:00:00+00:00,2,0.0,0.0000
32,sza,Positive,2021-01-16 00:00:00+00:00,8,0.0,0.0000
33,sza,Positive,2021-01-18 00:00:00+00:00,3,0.0,0.0000
34,sza,Positive,2021-01-19 00:00:00+00:00,2,0.0,0.0000
35,sza,Positive,2021-01-20 00:00:00+00:00,1,0.0,0.0000
36,sza,Positive,2021-01-21 00:00:00+00:00,8,0.0,0.0000
37,sza,Positive,2021-01-22 00:00:00+00:00,1,0.0,3.4375
38,sza,Positive,2021-01-23 00:00:00+00:00,11,0.0,3.8125
39,sza,Positive,2021-01-24 00:00:00+00:00,10,0.0,4.8125
40,sza,Positive,2021-01-25 00:00:00+00:00,3,0.0,5.5000


In [95]:
fig = px.bar(df_u, x="created_at", y="total", color="sentiment", barmode="group",category_orders=co, title=f'{user}: Tweets',hover_data=df_u.columns)
fig.add_trace(go.Scatter(x=df_u[df_u['sentiment']=='Negative']['created_at'],y=df_u[df_u['sentiment']=='Negative']['nroll_7'], name='nroll_7'))
fig.add_trace(go.Scatter(x=df_u[df_u['sentiment']=='Positive']['created_at'],y=df_u[df_u['sentiment']=='Positive']['proll_7'], name='proll_7'))

url = py.plot(fig, filename=f'{user}_bar')

In [15]:
print(url)

https://plotly.com/~arhanna/54/


#### Time Series With two bars per day of negative/positive sum

In [52]:
df_u.total.rolling(7,win_type='triang').mean()

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6     3.3125
7     3.3125
8     2.8750
9     2.4375
10    2.0625
11    1.8125
12    1.8125
13    1.8125
14    1.6875
15    1.6250
16    1.5625
17    1.6250
18    1.8125
19    2.0625
20    2.3750
21    2.3750
22    2.5625
23    3.1250
24    3.8750
25    5.0625
26    6.1250
27    6.5000
28    6.3750
29    5.8750
30    4.8125
31    3.9375
32    3.5625
33    3.1875
34    3.2500
35    3.6250
36    3.8750
37    4.5625
38    5.6250
39    6.2500
40    6.8125
41    7.6250
42    7.9375
43    8.6875
44    9.3125
45    9.0000
Name: total, dtype: float64